In [1]:
import cv2
import os
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [5]:
def extract_frames_from_video(video, target_fps=2):
    """
    Extract frames at a rate of 'target_fps' per second from a video object and return them in a list.
    :param video: A video capture object (cv2.VideoCapture).
    :param target_fps: Target number of frames per second to extract (e.g., 2 frames per second).
    :return: A list of frames from the video.
    """
    frames = []  # List to hold frames
    
    # Get the video's frame rate (frames per second)
    fps = video.get(cv2.CAP_PROP_FPS)
    # Calculate the frame interval for the target frames per second
    frame_interval = int(fps / target_fps)
    # Initialize a counter for frames
    count = 0
    
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        if not ret:
            break  # End of video
        # Only extract frames that match the target fps
        if count % frame_interval == 0:
            frames.append(frame)  
        # Increment the counter
        count += 1
    print(f"Extracted {len(frames)} frames.")
    return frames

In [7]:
def extract_highest_confidence_face_from_frames(frames):
    """
    Extract the face with the highest confidence score from a list of frames using MTCNN.
    :param frames: List of frames (images) extracted from the video.
    :return: List of the highest confidence face images from each frame.
    """
    # Initialize MTCNN face detector
    mtcnn = MTCNN()
    face_images = []  # List to hold cropped face images
    # Iterate over each frame
    for idx, frame in enumerate(frames):
        # Detect faces in the frame
        faces = mtcnn.detect_faces(frame)  
        # If faces are detected, pick the one with the highest confidence
        if faces:
            highest_confidence_face = None
            max_confidence = 0  # Variable to store the highest confidence  
            # Find the face with the highest confidence score
            for face in faces:
                confidence = face['confidence']
                if confidence > max_confidence:
                    max_confidence = confidence
                    highest_confidence_face = face   
            # Crop the face with the highest confidence from the frame
            if highest_confidence_face:
                x, y, w, h = highest_confidence_face['box']
                face_crop = frame[y:y+h, x:x+w]
                face_images.append(face_crop)
    print(f"Extracted {len(face_images)} faces with the highest confidence.")
    return face_images

In [9]:
def preprocess_image(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (224, 224))
    image_preprocessed = preprocess_input(image_resized) 
    image_batch = np.expand_dims(image_preprocessed, axis=0)

    return image_batch

In [11]:
def feature_extraction(faces):
    features = []

    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    feature_extractor = Model(inputs=model.input, outputs=model.output)

    for face in faces:
        face_n = preprocess_image(face)
        face_feature = feature_extractor.predict(face_n) 
        features.append(face_feature)

    features = np.concatenate(features, axis=0)

    features = np.expand_dims(features, axis=0)

    return features

In [17]:
# Load the saved model
model = load_model(r"C:\Users\wiame\Documents\RECONNAISSANCE DES FORMES\PROJET\DeepFake Detection\my_model.keras")

In [19]:
def prediction(video_capture):
    frames_list = extract_frames_from_video(video_capture, 2)
    faces = extract_highest_confidence_face_from_frames(frames_list)
    ResNet_features = feature_extraction(faces)
    max_length = 108 # Set this according to your data (e.g., max number of frames per sequence)
    features_padded = pad_sequences(ResNet_features, maxlen=max_length, dtype='float32', padding='post', truncating='post')
    predictions = model.predict(np.array(features_padded))
    pred = (predictions> 0.5).astype(int)
    if pred == 1 :
        print("This video is FAKE")
    elif pred == 0 : 
        print("This video is REAL")

    return predictions

In [21]:
video_path = r"C:\Users\wiame\Documents\RECONNAISSANCE DES FORMES\PROJET\DeepFake Detection\deepfaked_video.mp4"
video_capture = cv2.VideoCapture(video_path)
prediction(video_capture)

Extracted 42 frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

array([[0.9988457]], dtype=float32)